In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import os
import json

%matplotlib inline

In [2]:
## A dict with sources:
with open('sources.json') as data_file:    
    sources = json.load(data_file)

In [11]:
def get_series_from_mecd(url, args):

    # Here I'm truncating the documentID in mecd.gob.es; hope
    # it doesn't bite me.
    temp_file = "data/%s.xlsx" % url[-12:]
  
    # Don't download if we already have the file
    if not os.path.isfile(temp_file):
        urllib.urlretrieve(url, temp_file)
        
        
    # Squeeze to get a Series if only one column
    series = pd.read_excel(temp_file, squeeze=True, **args)    
    
    # A lot of random extra spaces in index columns
    series.index = series.index.str.strip()
    
    return series

In [12]:
series = {}

for name, values in sources.iteritems():
    url = values[0]
    args = values[1]
        
    series[name] = get_series_from_mecd(*values)
    
pd.DataFrame(series).head()

,math_level_1,math_level_5,math_level_6,math_score,math_under_1,reading_level_1,reading_level_5,reading_level_6,reading_score,reading_under_1,science_level_1,science_level_5,science_level_6,science_score,science_under_1
Alemania,12.193542,12.773757,4.681767,513.525056,5.545130,3.266599,24.577259,8.260698,507.676530,0.521725,9.312185,10.574336,1.590635,524.120799,2.902123
Andalucía,18.915585,4.732912,1.013611,472.019591,8.478546,5.178162,16.154161,3.984725,476.812111,1.643554,14.102856,4.116931,0.300497,486.034650,4.516763
Aragón,13.386463,9.498153,1.957469,496.266194,7.881926,4.429627,19.945232,5.932133,493.141800,1.096787,11.864777,6.762451,0.842837,504.111080,3.938644
Asturias,11.890078,10.387162,2.444741,499.601625,6.777473,3.674848,23.231226,7.373033,504.016976,1.305270,8.927664,8.123267,1.236010,516.861585,2.731494
Australia,13.534663,10.515042,4.294680,504.150766,6.132482,3.106353,23.336504,9.798264,511.803998,0.900302,10.214973,10.912487,2.644107,521.494746,3.430821


In [40]:
othersources = {
    "repeaters_countries" : ["http://dx.doi.org/10.1787/888932957403",
           {"sheetname": "Data_Figure IV.1.4_1",
            "index_col" : 0,
            "skiprows" : 8,
            "parse_cols" : "A,E",
            "skip_footer" : 68,
            "comments" : "Filas: 9 (encabezados) y países: 10 a 43. Columnas: A, E*100"
           }],
    "repeaters_regions" : ["http://dx.doi.org/10.1787/888932957536",
           {"sheetname": "Table B2.IV.1",
            "index_col" : 0,
            "skiprows" : 93,
            "parse_cols" : "A,T",
            "skip_footer" : 58,
            "comments" : "Filas: 9 (encabezados) y países: 10 a 43. Columnas: A, E*100"
           }]    
    
    
}

In [43]:
x = get_series_from_mecd(*othersources['repeaters_regions'])    
x

Spain
Andalusia*           36.738454
Aragon*              36.098311
Asturias*            27.438242
Balearic Islands*    39.145251
Basque Country*      20.761495
Cantabria*           32.284908
Castile and Leon*    34.515604
Catalonia*           20.599708
Extremadura*         42.863706
Galicia*             32.955194
La Rioja*            33.988697
Madrid*              32.397438
Murcia*              42.544068
Navarre*             25.288858
Name: Unnamed: 1, dtype: float64